In [2]:
import pandas as pd
import numpy as np
from ast import literal_eval
import re
import warnings
warnings.filterwarnings("ignore")

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [3]:
contents_pd = pd.read_csv('C:/Users/2hy03/Python_personal/Dx추천/kr_dx_concat_pre.csv', encoding='utf-8')
contents_pd.columns =['id', 'title', 'url', 'date', 'keyword', 'creator', 'view','likes', 'depart', 'depart1', 'language', 'text', 'new_text','key_words']

In [4]:
contents=contents_pd[[
    'id','title','date','creator','view','likes','depart','key_words'
]]

In [5]:
contents.head()

,id,title,date,creator,view,likes,depart,key_words
0,0,1강. 디지털 트랜스포메이션(DX)에 대한 이해,2021.5.7,KPC디지털,2508,29.0,DX이해,"디지털 트랜스포메이션, 생산성, 디지털 기술, 디지털 혁신, 산업DX, 디지털전환"
1,1,2강. 디지털 트랜스포메이션을 위한 사전준비,2021.5.7,KPC디지털,764,11.0,DX이해,"디지털 트랜스포메이션, 디지털 혁신, 산업 DX, DX 전담조직PMO, KPC 디지..."
2,2,3강. 디지털 역량 진단,2021.5.7,KPC디지털,799,9.0,DX이해,"디지털 역량 진단, 디지털 트랜스포메이션, 디지털 비전, 리더십, 디지털 전략, 디..."
3,3,4강. 목표수립 및 방향설정,2021.5.7,KPC디지털,504,9.0,DX이해,"디지털 트랜스포메이션, 디지털 비전, 디지털 리더십, 디지털 거버넌스, DX 교육,..."
4,4,5강. 디지털 비즈니스 모델 혁신,2021.5.7,KPC디지털,405,5.0,DX이해,"디지털 비즈니스 모델, 디지털 비즈니스 모델 혁신, 디지털 트랜스포메이션, 비즈니스..."


In [6]:
contents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 757 entries, 0 to 756
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         757 non-null    int64  
 1   title      757 non-null    object 
 2   date       757 non-null    object 
 3   creator    757 non-null    object 
 4   view       757 non-null    int64  
 5   likes      742 non-null    float64
 6   depart     757 non-null    object 
 7   key_words  757 non-null    object 
dtypes: float64(1), int64(2), object(5)
memory usage: 47.4+ KB


In [7]:
contents['likes'] = contents['likes'].fillna(0)

## TF-IDF기반 추천시스템
- key_words기반
- title기반
- title + key_words?

In [8]:
# 제목 기반
def tfidf_title_recommendation(contents, sample_title, top_n=5):

    sample_index = contents[contents['title'] == sample_title].index[0]

    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(contents['title'])

    similarity_scores = cosine_similarity(tfidf_matrix[sample_index], tfidf_matrix)

    top_indices = similarity_scores.argsort()[0][-top_n-1:-1][::-1]
    top_scores = similarity_scores[0][top_indices]
    top_contents = contents.iloc[top_indices]
    top_contents['similarity_score'] = top_scores

    return top_contents

# 키워드 기반
def tfidf_keyword_recommendation(contents, sample_title, top_n=5):
    sample_index = contents[contents['title'] == sample_title].index[0]

    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(contents['key_words'])

    similarity_scores = cosine_similarity(tfidf_matrix[sample_index], tfidf_matrix)

    top_indices = similarity_scores.argsort()[0][-top_n-1:-1][::-1]
    top_scores = similarity_scores[0][top_indices]
    top_contents = contents.iloc[top_indices]
    top_contents['similarity_score'] = top_scores

    return top_contents


In [9]:
sample_title = input("샘플 컨텐츠 제목을 입력하세요: ")
recommended_contents = tfidf_keyword_recommendation(contents, sample_title)
recommended_contents = recommended_contents[recommended_contents['similarity_score'] > 0]  # 유사도가 0보다 큰 값만
recommended_contents[['title', 'creator','depart','key_words','similarity_score']]


샘플 컨텐츠 제목을 입력하세요: 사례로 알아보는 AI 활용한 RPA고도화


,title,creator,depart,key_words,similarity_score
386,"자동화: 마케팅, 영업, 서비스를 더 쉽게 만드는 방법 | Salesforce Ex...",세일즈포스 - Salesforce Korea,자동화기술,"자동화, 마케팅 자동화, 영업 자동화, 고객 서비스 자동화, Salesforce F...",0.302995
395,자동화로 혁신하는 제조산업,"헬로티_매일 만나는 산업, IT News",자동화기술,"자동화, 제조산업, RPA, 제조 자동화, 비즈니스 프로세스 혁신, 서비스 혁신, ...",0.277581
383,1 자피어Zapier 소개 - 24시간 온라인자동화 하는 방법! (부제 : 재피어...,강사자판기,자동화기술,"자동화, 온라인 자동화, 업무 효율, 온라인 마케팅, 솔루션, A/B 서비스 연결.",0.219590
111,디지털 트랜스포메이션의 시작 : RPA + 프로세스 마이닝,TalkIT,DX이해,"디지털 트랜스포메이션, RPA, 프로세스 마이닝, 업무 자동화, 비효율성, 컴플라이...",0.203198
380,"[웨비나 다시보기] 매출 성장의 핵심, 오라클 마케팅 자동화 고객 사례",Oracle Korea,자동화기술,"웨비나, 매출 성장, 마케팅 자동화, 오라클, 실시간 마케팅, 클라우드, 마케팅 자동화",0.193280


In [10]:
sample_title = input("샘플 컨텐츠 제목을 입력하세요: ")
recommended_contents = tfidf_title_recommendation(contents, sample_title)
recommended_contents = recommended_contents[recommended_contents['similarity_score'] > 0]  # 유사도가 0보다 큰 값만
recommended_contents[['title', 'creator','depart','key_words','similarity_score']]

샘플 컨텐츠 제목을 입력하세요: 사례로 알아보는 AI 활용한 RPA고도화


,title,creator,depart,key_words,similarity_score
260,10분으로 알아보는 AI 기술의 적용 사례들 알아보기! | 십분 AI,십분AI,데이터기술,"AI 기술, 적용 사례, 디지털 혁신",0.263144
259,10분으로 알아보는 빅데이터 기술 & 사례들! | 십분AI 마지막 강의,십분AI,데이터기술,"빅데이터 기술, 사례, AI (인공지능), 데이터 분석, 데이터 시각화, ...",0.164828
88,4. 해외 사례로 배우는 성공적인 D2C 마케팅/고객경험 전략 | 채널코퍼레이션 김...,바이라인네트워크 비즈니스 테크TV,DX이해,"D2C, 마케팅, 고객경험, 제조 브랜드, 온라인, 코로나19, 글로벌, 자사몰, ...",0.161886
194,Doing Agile과 Being Agile로 알아보는 애자일 도입 전략,투이컨설팅-투이톡,DX태도,"애자일, 도입 전략, 실패 이유, 디지털 비즈니스, 디지털 변화, 디지털 혁신, 디...",0.157369
257,KNIME을 활용한 중고차 예측모델,HJ,데이터기술,"KNIME, 머신러닝, 중고차, 예측모델, 데이터 분석, 데이터 모델링.",0.151555
